---
# Database
---

In [1]:
# clonen
!git clone https://github.com/sleuoth/ABDA2019.git
!ls ABDA2019/testdaten/cryptominuteresolution
print()

# entpacken 'btcusd'
!unzip ABDA2019/testdaten/cryptominuteresolution/btcusd.csv.zip
!mv btcusd.csv ABDA2019/testdaten/cryptominuteresolution/btcusd.csv

fatal: destination path 'ABDA2019' already exists and is not an empty directory.
abseth.csv	cndbtc.csv  foausd.csv	mkrdai.csv  rdnusd.csv	utkbtc.csv
absusd.csv	cndeth.csv  fsnbtc.csv	mkreth.csv  repbtc.csv	utketh.csv
agibtc.csv	cndusd.csv  fsneth.csv	mkrusd.csv  repeth.csv	utkusd.csv
agieth.csv	cnneth.csv  fsnusd.csv	mlneth.csv  repusd.csv	utneth.csv
agiusd.csv	cnnusd.csv  fttusd.csv	mlnusd.csv  reqbtc.csv	utnusd.csv
aidbtc.csv	csxeth.csv  fttust.csv	mnabtc.csv  reqeth.csv	veebtc.csv
aideth.csv	csxusd.csv  funbtc.csv	mnaeth.csv  requsd.csv	veeeth.csv
aidusd.csv	ctxbtc.csv  funeth.csv	mnausd.csv  rifbtc.csv	veeusd.csv
aiobtc.csv	ctxeth.csv  funusd.csv	mtnbtc.csv  rifusd.csv	vetbtc.csv
aioeth.csv	ctxusd.csv  geneth.csv	mtneth.csv  rlcbtc.csv	veteth.csv
aiousd.csv	dadbtc.csv  genusd.csv	mtnusd.csv  rlceth.csv	vetusd.csv
algbtc.csv	dadeth.csv  gnoeth.csv	ncabtc.csv  rlcusd.csv	vldeth.csv
algusd.csv	dadusd.csv  gnousd.csv	ncaeth.csv  rrbusd.csv	vldusd.csv
algust.csv	daibtc.csv  gntbtc.csv	n

---
# Frameworkinstallation
---

In [2]:
# jdk
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# tree
!apt-get install tree

# spark-package
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
#!tree spark-2.4.4-bin-hadoop2.7

# findspark
!pip install findspark

# numpy
!pip install numpy

# timeseries library
!pip install ts ts-flint #!pip3 install ta-lib --force-reinstall wheel datascience urllib3 kaggle

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tree is already the newest version (1.7.0-5).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


---
# Code
*@see [Investopedia.com](https://www.investopedia.com/terms/p/ppo.asp#formula-and-calculation-for-ppo)*

---

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

if 'COLAB_TPU_ADDR' not in os.environ:
  print(">> Colab connected to TPU")
else:
  print(">> Colab not connected to TPU")
  
import findspark

import matplotlib.pyplot as plt

findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext

#spark = SparkSession.builder.config('spark.driver.extraClassPath', '/path/to/postgresql.jar').getOrCreate()

spark = SparkSession.builder.master("local[*]").getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

>> Colab connected to TPU


In [0]:
preview_limit = 5  #@param {type: "slider", min: 1, max: 100}
dataframe_from = "Git" #@param ["Git", "Sample: Intel (requieres './intc.csv')"] {allow-input: false}
filename_to_load = 'btcusd.csv'  #@param {type: "string"}



*Metadaten-Aggregation*
---
---

In [5]:
from pyspark.sql.functions import input_file_name

if dataframe_from == "Git":
  df_spark = spark.read.csv('./ABDA2019/testdaten/cryptominuteresolution/'+filename_to_load, inferSchema=True, header=True)
  start = 59
  end = 69

elif dataframe_from == "Sample: Intel (requieres './intc.csv')":
  df_spark = spark.read.csv('./intc.csv', inferSchema=True, header=True)
  start = 17
  end = 26
  # For more info about this example see
  # https://school.stockcharts.com/doku.php?id=technical_indicators:price_oscillators_ppo


df_spark = df_spark \
  .withColumn('filepath', input_file_name()) \
  .withColumn('filename', input_file_name()[start:end]) \
  .withColumn('timestamp', df_spark['time']/1000) \
  .withColumn('date', (df_spark['time']/1000).cast('timestamp'))

print("Type:", type(df_spark), "\n")
print("Schema: ", end = '')
df_spark.printSchema()
df_spark_total_count = df_spark.count()
print("Available rows:", df_spark_total_count, "\n")

df_spark.show(preview_limit)

df_spark.createOrReplaceTempView("base_data")

df_spark_informationView = spark.sql("SELECT filename, count(*), min(date), max(date)  FROM base_data group by filename")
df_spark_informationView.show()

Type: <class 'pyspark.sql.dataframe.DataFrame'> 

Schema: root
 |-- time: long (nullable = true)
 |-- open: double (nullable = true)
 |-- close: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- filepath: string (nullable = false)
 |-- filename: string (nullable = false)
 |-- timestamp: double (nullable = true)
 |-- date: timestamp (nullable = true)

Available rows: 2457265 

+-------------+-----+-----+-----+-----+-----------+--------------------+----------+------------+-------------------+
|         time| open|close| high|  low|     volume|            filepath|  filename|   timestamp|               date|
+-------------+-----+-----+-----+-----+-----------+--------------------+----------+------------+-------------------+
|1364774820000|93.25| 93.3| 93.3|93.25|       93.3|file:///content/A...|btcusd.csv|1.36477482E9|2013-04-01 00:07:00|
|1364774880000|100.0|100.0|100.0|100.0|       93.3|file:///conte

*Tag-Filter*
---
---

In [6]:
#
# TAG-FILTER
#

base_data_dayfiltered = spark.sql(
  """SELECT d.* FROM base_data d  
     where  (d.date, filename) in (select max(m.date), filename from base_data m group by date_format(m.date, "y-M-d"), filename)
  """)
base_data_dayfiltered.createOrReplaceTempView("base_data_dayfiltered")
#base_data_dayfiltered.show(preview_limit)

base_data_dayfiltered_total_count = base_data_dayfiltered.count()
print("Available rows:", base_data_dayfiltered_total_count, "(lost",(df_spark_total_count-base_data_dayfiltered_total_count), "rows)", "\n")

base_data_dayfiltered_informationView = spark.sql("SELECT filename, count(*), min(date), max(date)  FROM base_data_dayfiltered group by filename")
base_data_dayfiltered_informationView.show()

Available rows: 2211 (lost 2455054 rows) 

+----------+--------+-------------------+-------------------+
|  filename|count(1)|          min(date)|          max(date)|
+----------+--------+-------------------+-------------------+
|btcusd.csv|    2211|2013-04-01 23:58:00|2019-10-01 18:46:00|
+----------+--------+-------------------+-------------------+



*EMA-UDF*
---
---

In [0]:
#
# EMA-UDF
#

from pyspark.sql.functions import col, collect_list, concat_ws, udf
from pyspark.sql.types import DoubleType

# see https://tradistats.com/exponentieller-gleitender-durchschnitt/
def ema(ar):
    if len(ar) > 0:
       SF  = 2/ (len(ar)+1)
       SFi = 1 - SF
       my_ema = ar[0]
       for i in ar:
           my_ema = (i * SF) + (my_ema * SFi)
    return my_ema

ema_udf = udf(ema, DoubleType())

*Windows*
---
---

In [0]:
#
# EMA-WINDOWS
#

from pyspark.sql.window import Window

win26 = (Window
    .partitionBy("filename")
    .orderBy("date") \
    .rowsBetween(-25, 0))

win12 = (Window
    .partitionBy("filename")
    .orderBy("date") \
    .rowsBetween(-11, 0))

win9 = (Window
    .partitionBy("filename")
    .orderBy("date") \
    .rowsBetween(-8, 0))

*PPO- & Signal-Berechnung*
---
---

In [0]:
#
# EMA_26 && EMA_12 CALCULATION
#

base_data_tmp_ema_1 = base_data_dayfiltered \
  .withColumn('win26_close_list', collect_list('close').over(win26)) \
  .withColumn('win12_close_list', collect_list('close').over(win12))

base_data_tmp_ema_2 = base_data_tmp_ema_1.select(
  "*",
  ema_udf(base_data_tmp_ema_1["win26_close_list"]).alias("EMA26"),
  ema_udf(base_data_tmp_ema_1["win12_close_list"]).alias("EMA12")
)

base_data_tmp_ema_2.createOrReplaceTempView("base_data_tmp_ema_2")
#base_data_tmp_ema_2.show(preview_limit)

In [0]:
#
# PPO BERECHNUNG
#

base_data_tmp_ema_3 = spark.sql(
  " select *, (((EMA12 - EMA26) / EMA26) * CAST(100 AS DOUBLE))   " \
  " FROM base_data_tmp_ema_2 " \
  " order by filename asc, date desc "
)

#
# SIGNAL BERECHNUNG
#
base_data_tmp_ema_4 = base_data_tmp_ema_3 \
  .withColumn('win9_ema12sub26_list', collect_list('(((EMA12 - EMA26) / EMA26) * CAST(100 AS DOUBLE))').over(win9))


base_data_tmp_ema_5 = base_data_tmp_ema_4.select(
  "*",
  base_data_tmp_ema_4['(((EMA12 - EMA26) / EMA26) * CAST(100 AS DOUBLE))'].alias("PPO"),
  ema_udf(base_data_tmp_ema_4["win9_ema12sub26_list"]).alias("SIGNAL")
)

base_data_tmp_ema_5.createOrReplaceTempView("base_data_tmp_ema_5")
#base_data_tmp_ema_5.show(preview_limit)

*PPO_Histogram-Berechnung*
---
---

In [0]:
#
# PPO-HISTOGRAM BERECHNUNG
#

base_data_tmp_ema_6 = spark.sql(
  " select *, (PPO - SIGNAL)   " \
  " FROM base_data_tmp_ema_5 " \
  " order by filename asc, date desc "
)

base_data_ema = base_data_tmp_ema_6.select(
  "*",
  base_data_tmp_ema_6['(PPO - SIGNAL)'].alias("PPO_HISTOGRAM")
)

base_data_ema.createOrReplaceTempView("base_data_ema")
#base_data_ema.show(preview_limit)

---
# Analyse 
---

*@see [Investopedia.com](https://www.investopedia.com/terms/p/ppo.asp#what-the-indicator-tells-you)*

**→ ppo_trend_analysis** :

Auf den 'PPO' angewendet (window mind. 1):

> *When the PPO is above zero, that helps confirm an uptrend since the short-term EMA is above the longer-term EMA. When the PPO is below zero, the short-term EMA is below the longer-term EMA, which is an indication of a downtrend.*


**→ ppo_crossover_signal_analysis** :

Auf den 'PPO_HISTOGRAM' angewendet (window mind. 2):
> *The indicator generates a buy signal when the PPO line crosses above the signal line from below, and a sell signal occurs when the PPO line crosses  below the signal from above.*

Auf den 'PPO' angewendet (window mind. 2):
> *Centerline crossovers also generate trading signals. Traders consider a move from below to above the centerline as bullish, and a move from above to below the centerline as bearish.*

**→ ppo_technical_divergence_analysis:**

Auf den 'close' und 'PPO' (window mind. 2; ggf. 'close' mit 'open' tauschen):
> *Traders can also use the PPO to look for technical divergence between the indicator and price. For example, if the price of an asset makes a higher high, but the indicator makes a lower high, it may indicate the upward momentum is subsiding. Conversely, if an asset's price makes a lower low, but the indicator makes a higher low, it could suggest that the bears are losing their traction and the price could head higher soon.*
---


*Windows*
---
---

In [0]:
#
# Analyse-Windows
#

from pyspark.sql.window import Window

win2 = (Window
    .partitionBy("filename")
    .orderBy("date") \
    .rowsBetween(-1, 0))

win3 = (Window
    .partitionBy("filename")
    .orderBy("date")
    .rowsBetween(-2, 0))

*Funktionen*
---
---

In [0]:
#
# Analyse-UDF
#

from pyspark.sql.types import StringType

def ppo_trend_analysis(ar):
    output = ""
    count = len(ar)
    if count > 0:
      # Einzelabgleich
      if count == 1:
        current = ar[0]
        if current > 0:
          output = "▲ Aufwärtstrend" # "Aufwärtstrend"
        elif current < 0:
          output = "▼ Abwärtstrend" # "Abwärtstrend"

      # Vergleich mit den davorherigenden; ggf. mit mehreren vorherigen
      elif count > 1:
        last_position = count - 1
        next_to_last_position = last_position - 1

        last = ar[last_position] 
        next_to_last = ar[next_to_last_position]

        if last > 0 and next_to_last > 0:
          output = "▲ Aufwärtstrend" # "Solider Uptrend"
        elif last < 0 and next_to_last > 0:
          output = "↘ neuer Abwärtstrend" # "Emerging Downtrend"
        elif last < 0 and next_to_last < 0:
          output = "▼ Abwärtstrend" # "Solid Downtrend"
        elif last > 0 and next_to_last < 0:
          output = "↗ neuer Aufwärtstrend" # "Emerging Uptrend"

    return output

def ppo_crossover_signal_analysis(ar):
    output = ""
    count = len(ar)
    # Vergleich des letzten Crossovers
    if count > 1:
        last_position = count - 1
        next_to_last_position = last_position - 1
        
        last = ar[last_position] 
        next_to_last = ar[next_to_last_position]

        if last <= 0 and next_to_last > 0:
          output = "Verkauf-Signal" # "Verkauf Signal"
        elif last >= 0 and next_to_last < 0:
          output = "Kauf-Signal" # "Kauf Signal"

    return output

def ppo_technical_divergence_analysis(close_ar, ppo_ar):
    output = ""
    count_close = len(close_ar)
    count_ppo = len(ppo_ar)
    
    # Arrays müssen selbe Größe haben
    if count_close == count_ppo and count_close > 1:
      # delta berechnen
      pointer = count_close - 1
      delta_close = close_ar[pointer]
      delta_ppo = ppo_ar[pointer]
      pointer -= 1

      while pointer >= 0:
        delta_close -= close_ar[pointer]
        delta_ppo -= ppo_ar[pointer]
        pointer -= 1

      # steigung berechnen
      div_close = delta_close / count_close
      div_ppo = delta_ppo / count_ppo

      if div_close > div_ppo:
        output = "↧ Abflachend"
      else:
        output = "↥ Wachsend"

    return output

ppo_trend_analysis_udf = udf(ppo_trend_analysis, StringType())
ppo_crossover_signal_analysis_udf = udf(ppo_crossover_signal_analysis, StringType())
ppo_technical_divergence_analysis_udf = udf(ppo_technical_divergence_analysis, StringType())

*Aggregation*
---
---

In [0]:
#
# SIGNAL ANALYSIS
#

base_data_ema_tmp_eval = base_data_ema \
  .withColumn('win2_ppo_list', collect_list('PPO').over(win2)) \
  .withColumn('win2_ppoh_list', collect_list('PPO_Histogram').over(win2)) \
  .withColumn('win2_close_list', collect_list('close').over(win2))

base_data_ema_eval_result = base_data_ema_tmp_eval.select(
  "*",
  ppo_trend_analysis_udf(base_data_ema_tmp_eval["win2_ppo_list"]).alias("TREND"),
  ppo_crossover_signal_analysis_udf(base_data_ema_tmp_eval["win2_ppo_list"]).alias("WEAK_SIGNAL"),
  ppo_crossover_signal_analysis_udf(base_data_ema_tmp_eval["win2_ppoh_list"]).alias("STRONG_SIGNAL"),
  ppo_technical_divergence_analysis_udf(base_data_ema_tmp_eval["win2_close_list"], base_data_ema_tmp_eval["win2_ppo_list"]).alias("DIVERGENCE")
)

base_data_ema_eval_result.createOrReplaceTempView("base_data_ema_eval_result")
#base_data_ema_eval_result.show(preview_limit)

---
# Auswertung
---

*Für alle finale Empfehlungen gilt eine Reichteweite von 0 (schwach) bis 3 (stark)*

**Halten**
---
---
*   **H0:** 
  * ▼ Abwärtstrend && ↧ Abflachend
  * ▼ Abwärtstrend && ↥ Wachsend
  * ↘ neuer Abwärtstrend && ↧ Abflachend
*   **H1:**
  * ↗ neuer Aufwärtstrend && ↧ Abflachend
*   **H2:**
  * ▲ Aufwärtstrend && ↧ Abflachend
  * ↘ neuer Abwärtstrend && ↥ Wachsend
*   **H3:**
  * ▲ Aufwärtstrend && ↥ Wachsend
  * ↗ neuer Aufwärtstrend && ↥ Wachsend

In [0]:
#
# Halten-SQL-Statement
#

holdingCaseStatement = "CASE WHEN (TREND = \"↗ neuer Aufwärtstrend\" AND DIVERGENCE = \"↧ Abflachend\") THEN \"H1\"" +"\n"+\
"WHEN (TREND = \"▲ Aufwärtstrend\" = \"↧ Abflachend\") OR (TREND = \"↘ neuer Abwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\") THEN \"H2\"" +"\n"+\
"WHEN (TREND = \"▲ Aufwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\") OR (TREND = \"↗ neuer Aufwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\") THEN \"H3\"" +"\n"+\
"ELSE \"H0\" END" +"\n"

**Kaufen**
---
---
*   **K0:** 
  * ▼ Abwärtstrend && ↧ Abflachend
  * ↘ neuer Abwärtstrend && ↧ Abflachend
*   **K1:**
  * ▼ Abwärtstrend && ↥ Wachsend
  * ↘ neuer Abwärtstrend && ↥ Wachsend
  * ↗ neuer Aufwärtstrend && ↧ Abflachend
*   **K2:**
  * ▲ Aufwärtstrend && ↧ Abflachend
  * ↗ neuer Aufwärtstrend && ↥ Wachsend
*   **K3:**
  * ▲ Aufwärtstrend && ↥ Wachsend

In [0]:
#
# Kaufen-SQL-Statement
#

buyingCaseStatement = "CASE WHEN (TREND = \"▼ Abwärtstrend\" AND DIVERGENCE = \"↧ Abflachend\") OR (TREND = \"↘ neuer Abwärtstrend\" AND DIVERGENCE = \"↧ Abflachend\") THEN \"K0\"" +"\n"+\
"WHEN (TREND = \"▲ Aufwärtstrend\" AND DIVERGENCE = \"↧ Abflachend\") OR (TREND = \"↗ neuer Aufwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\") THEN \"K2\"" +"\n"+\
"WHEN (TREND = \"▲ Aufwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\") THEN \"K3\"" +"\n"+\
"ELSE \"K1\" END" +"\n"

**Verkaufen**
---
---
*   **V0:** 
  * ▲ Aufwärtstrend && ↥ Wachsend
  * ▲ Aufwärtstrend && ↧ Abflachend
*   **V1:**
  * ↗ neuer Aufwärtstrend && ↥ Wachsend
  * ↘ neuer Abwärtstrend && ↥ Wachsend
*   **V2:**
  * ▼ Abwärtstrend && ↥ Wachsend
  * ↗ neuer Aufwärtstrend && ↧ Abflachend
  * ↘ neuer Abwärtstrend && ↧ Abflachend
*   **V3:**
  * ▼ Abwärtstrend && ↧ Abflachend

In [0]:
#
# Verkaufen-SQL-Statement
#

sellingCaseStatement = "CASE WHEN (TREND = \"▲ Aufwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\")OR (TREND = \"▲ Aufwärtstrend\" AND DIVERGENCE = \"↧ Abflachend\") THEN \"V0\"" +"\n"+\
"WHEN (TREND = \"↗ neuer Aufwärtstrend\" = \"↥ Wachsend\" AND DIVERGENCE = \"↥ Wachsend\") OR (TREND = \"↘ neuer Abwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\") THEN \"V1\"" +"\n"+\
"WHEN (TREND = \"▼ Abwärtstrend\" AND DIVERGENCE = \"↧ Abflachend\") THEN \"V3\" " +"\n"+\
"ELSE \"V2\" END" +"\n"

**Aggregation**
---
---

* Wenn beide Signale null sind ⇒ *halten*
* Ansonsten 'Strong-' > 'Weaksignal', für jeweils beide gilt:
  * KAUF-Signal ⇒ *kaufen*
  * VERKAUF-Signal ⇒ *verkaufen*

```
+----------------+----------------+------------+
|  WEAK_SIGNAL   | STRONG_SIGNAL  | EMPFEHLUNG |
+----------------+----------------+------------+
| null           | null           | HALTEN     |
| Kauf-Signal    | null           | KAUFEN     |
| Kauf-Signal    | Kauf-Signal    | KAUFEN     |
| ...            | ...            | ...        |
| Verkauf-Signal | null           | VERKAUFEN  |
| Verkauf-Signal | Verkauf-Signal | VERKAUFEN  |
| ...            | ...            | ...        |
| Kauf-Signal    | Verkauf-Signal | VERKAUFEN  |
| Verkauf-Signal | Kauf-Signal    | KAUFEN     |
+----------------+----------------+------------+
```


In [0]:
#
# Combiniertes-SQL-Statement
#

final_result_statement = "SELECT *, "  +"\n"+\
"CASE" +\
" " + "WHEN (WEAK_SIGNAL = \"\" AND STRONG_SIGNAL = \"\") THEN " + holdingCaseStatement +"\n"+\
" " + "WHEN (STRONG_SIGNAL = \"\") THEN " +"\n"+\
" " + "" + "CASE" +"\n"+\
" " + "" + "" + "WHEN (WEAK_SIGNAL = \"Kauf-Signal\") THEN " + buyingCaseStatement +"\n"+\
" " + "" + "" + "ELSE " + sellingCaseStatement +"\n"+\
" " + "" + "END" +"\n"+\
" " + "ELSE" +"\n"+\
" " + "" + "CASE" +"\n"+\
" " + "" + "" + "WHEN (STRONG_SIGNAL = \"Kauf-Signal\") THEN " + buyingCaseStatement +"\n"+\
" " + "" + "" + "ELSE " + sellingCaseStatement +"\n"+\
" " + "" + "END" +"\n"+\
" END AS PPO_RESULT" +"\n"+\
" FROM base_data_ema_eval_result"

data_final_eval_result = spark.sql(final_result_statement)
data_final_eval_result.createOrReplaceTempView("data_final_eval_result")

---
# Visualisierung
---

*Aktuell für nur einen Chart gedacht!*

In [0]:
#
# Imports
#

import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display
from IPython.display import HTML

In [20]:
base_data_dayfiltered_informationView.show()

+----------+--------+-------------------+-------------------+
|  filename|count(1)|          min(date)|          max(date)|
+----------+--------+-------------------+-------------------+
|btcusd.csv|    2211|2013-04-01 23:58:00|2019-10-01 18:46:00|
+----------+--------+-------------------+-------------------+



*Parameter*
---

In [0]:
display_limit = 365  #@param {type: "slider", min: 1, max: 10000}
display_accurarcy = 3  #@param {type: "slider", min: 1, max: 10}
template = "plotly_dark" #@param ['ggplot2', 'seaborn', 'simple_white', 'plotly', 'plotly_white', 'plotly_dark', 'presentation', 'xgridoff', 'ygridoff', 'gridon', 'none'] {allow-input: false}
startdate = '2018-10-01' #@param {type:"date"}
enddate = '2019-10-01' #@param {type:"date"}

display_accurarcy_str = str(display_accurarcy)

*Vorbereitung*
---

In [0]:
#
# Datavorbereitung
#

data_eval_result = spark.sql(
  " select Date(date), high, low, format_number(open, "+display_accurarcy_str+") open, format_number(close, "+display_accurarcy_str+") close" \
  " , format_number(PPO, "+display_accurarcy_str+") PPO, format_number(SIGNAL, "+display_accurarcy_str+") Signal, format_number(PPO_HISTOGRAM, "+display_accurarcy_str+") PPO_HISTOGRAM " \
  " , TREND, WEAK_SIGNAL, STRONG_SIGNAL, DIVERGENCE, PPO_RESULT "\
  " FROM data_final_eval_result " \
  " WHERE date >= '"+ startdate.strip() +" 00:00:00' AND date <= '"+ enddate.strip() +"23:59:59' " \
  " order by filename asc, date asc "
)

data_pd = data_eval_result.toPandas()

data_pd.close = (data_pd['close'].replace('\,','', regex = True).astype(float))
data_pd.open = (data_pd['open'].replace('\,','', regex = True).astype(float))

data_pd.date = pd.to_datetime(data_pd.date)
data_pd.close = pd.to_numeric(data_pd.close)
data_pd.high = pd.to_numeric(data_pd.high)
data_pd.low = pd.to_numeric(data_pd.low)
data_pd.PPO = pd.to_numeric(data_pd.PPO)
data_pd.Signal = pd.to_numeric(data_pd.Signal)
data_pd.PPO_HISTOGRAM = pd.to_numeric(data_pd.PPO_HISTOGRAM)

In [23]:
#
# Charts & Table
#

stock_chart = go.Figure(
    data = [ go.Candlestick( 
        x = data_pd['date'],
        open = data_pd['open'],
        high = data_pd['high'],
        low = data_pd['low'],
        close = data_pd['close']
        )]
)

ppo_chart = go.Figure(
    data = [ go.Scatter(
        x = data_pd['date'],
        y = data_pd['PPO'],
        mode = 'lines+markers',
        name = 'PPO'
    ), go.Scatter(
        x = data_pd['date'],
        y = data_pd['Signal'],
        mode = 'lines+markers',
        name = 'Signal'
    )]
)

ppo_chart.add_trace(
    go.Bar(
        name = 'PPO-Histogram',
        x = data_pd['date'],
        y = data_pd['PPO_HISTOGRAM']
    )
)

stock_chart.update_layout(
    title='Chart für CSV-Datei \'' + filename_to_load + '\'',
    yaxis_title = 'Points',
    template = 'plotly_dark',
    xaxis_rangeslider_visible = False
)

ppo_chart.update_layout(
    title='PPO-Indikator für CSV-Datei \'' + filename_to_load + '\'',
    yaxis_title = 'Changes in %',
    template = 'plotly_dark'
)

*Output*
---

In [24]:
print()
print("Zeitraum: "+startdate+" → "+enddate)
print()
stock_chart.show()
ppo_chart.show()
print()
display(HTML(data_pd.to_html(index = False, max_rows = display_limit)))


Zeitraum: 2018-10-01 → 2019-10-01



---
# Vergleichbare Charts
---

**BTCUSD**: [traidingview.com](https://de.tradingview.com/chart/ecCxGiMv/) *(login erforderlich)*

```
+-------------+------------+------------+-------------+-------------+
| TraidingView.com:                                                 |
+-------------+------------+------------+-------------+-------------+
|             | 2019-10-01 | 2019-09-30 | 2019-08-10  | 2019-08-07  |
+-------------+------------+------------+-------------+-------------+
| EMA26       |    9305.74 |    9384.68 |    10848.14 |    10630.29 |
| EMA12       |    8734.78 |    8810.38 |    11193.74 |    10886.97 |
| PPO         |      -6.14 |      -6.12 |        3.19 |        2.41 |
| SIGNAL      |      -4.56 |      -4.17 |        1.63 |        0.13 |
| PPO_History |      -1.57 |      -1.95 |        1.55 |        2.28 |
+-------------+------------+------------+-------------+-------------+

+-------------+------------+------------+-------------+-------------+
| CALCULATED:                                                       |
+-------------+------------+------------+-------------+-------------+
|             | 2019-10-01 | 2019-09-30 | 2019-08-10  | 2019-08-07  |
+-------------+------------+------------+-------------+-------------+
| EMA26       |  9327.3017 |  9446.6421 |  10651.5067 |  10673.4093 |
| EMA12       |  8751.5960 |  8842.4748 |  11163.8865 |  10785.1409 |
| PPO         |     -6.172 |    -6.3956 |      4.8104 |      1.0468 |
| SIGNAL      |    -4.4147 |    -4.0178 |      0.9102 |     -1.8162 |
| PPO_History |    -1.7576 |    -2.3778 |      3.9002 |      2.8631 |
+-------------+------------+------------+-------------+-------------+
```



**intc.csv (utf-8)** *--> eher ungeeignetes Beispiel*

 see [School.StockCharts.com](https://school.stockcharts.com/doku.php?id=technical_indicators:price_oscillators_ppo) 

```
time,open,close,high,low,volume
1271203140000,21.16,20.16,20.16,20.16,0.0
1271289540000,20.49,20.49,20.49,20.49,0.0
1271375940000,20.74,20.74,20.74,20.74,0.0
1271462340000,20.77,20.77,20.77,20.77,0.0
1271721540000,20.53,20.53,20.53,20.53,0.0
1271807940000,19.61,19.61,19.61,19.61,0.0
1271894340000,20.02,20.02,20.02,20.02,0.0
1271980740000,19.70,19.70,19.70,19.70,0.0
1272067140000,19.94,19.94,19.94,19.94,0.0
1272326340000,19.62,19.62,19.62,19.62,0.0
1272412740000,19.11,19.11,19.11,19.11,0.0
1272499140000,19.32,19.32,19.32,19.32,0.0
1272585540000,19.61,19.61,19.61,19.61,0.0
1272671940000,19.54,19.54,19.54,19.54,0.0
1272931140000,18.89,18.89,18.89,18.89,0.0
1273017540000,19.33,19.33,19.33,19.33,0.0
1273103940000,19.21,19.21,19.21,19.21,0.0
1273190340000,19.51,19.51,19.51,19.51,0.0
1273276740000,19.55,19.55,19.55,19.55,0.0
1273535940000,19.92,19.92,19.92,19.92,0.0
1273622340000,20.29,20.29,20.29,20.29,0.0
1273708740000,20.58,20.58,20.58,20.58,0.0
1273795140000,20.52,20.52,20.52,20.52,0.0
1273881540000,20.69,20.69,20.69,20.69,0.0
1274140740000,20.67,20.67,20.67,20.67,0.0
1274227140000,20.72,20.72,20.72,20.72,0.0
1274313540000,20.25,20.25,20.25,20.25,0.0
1274399940000,20.56,20.56,20.56,20.56,0.0
1274486340000,20.49,20.49,20.49,20.49,0.0
1274745540000,20.39,20.39,20.39,20.39,0.0

```

